In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
tf.config.list_physical_devices('GPU')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
!pip install tensorflow

In [4]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 5.4 MB/s eta 0:00:0000:010:01


In [7]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 MB 5.6 MB/s eta 0:00:0000:0100:01


In [9]:
!pip install pytorch_tabnet

  Using cached pytorch_tabnet-3.1.1-py3-none-any.whl (39 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 4.6 MB/s eta 0:00:0000:0100:01
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)


In [2]:

import pickle 
##Load data
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('/Users/abhisheksinghal/Documents/garima/MLLearning/ubi/supp.csv')

df.head()

y = df.target
x = df
x = x.drop(['target', 'row_id'], axis=1)
print(x.shape)
print(y.shape)
filename = "/Users/abhisheksinghal/Documents/garima/MLLearning/ubi/mar_model.sav"
final = "/Users/abhisheksinghal/Documents/garima/MLLearning/ubi/final_model.sav"


import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F



(942961, 302)
(942961,)


In [3]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

tabnet_params = dict(n_d=24, n_a=24, n_steps=1, gamma=1.3,
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )
regressor = TabNetRegressor(**tabnet_params)

# load the model from disk
regressor = pickle.load(open(filename, 'rb'))



Device used : cpu


In [3]:
tscv = TimeSeriesSplit(n_splits = 20)
rmse = []
CV_score_array    =[]
predictions_array =[]

for train_index, test_index in tscv.split(x):
    X_train, X_valid = x.iloc[train_index], x.iloc[test_index]
    y_train, y_valid = y[train_index], y[test_index]

    X_train      = X_train.to_numpy()
    y_train = y_train.to_numpy().reshape(-1, 1)
    
    X_valid      = X_valid.to_numpy()
    y_valid = y_valid.to_numpy().reshape(-1, 1)
    
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=300, max_epochs=100,
              eval_metric=['rmse'])

    predictions = regressor.predict(X_valid)
    pickle.dump(regressor, open(final, 'wb'))
    CV_score_array.append(regressor.best_cost)
    print("The CV score is %.5f" % np.mean(CV_score_array,axis=0) )
    predictions_array.append(np.expm1(predictions))
    
pickle.dump(regressor, open(final, 'wb'))

NameError: name 'TimeSeriesSplit' is not defined